In [ ]:
from folktables import ACSDataSource, BasicProblem, generate_categories
import numpy as np
import scipy.optimize as opt

### 1. Load and Preprocess the data
We are going to work with the [Folktables](https://github.com/socialfoundations/folktables#quick-start-examples) dataset (*you have already worked with it*).

1. As last week, we are still predicting the *Total person's income*  (I've digitized  it in  `target_transform=lambda x: x > 25000`).
2. Today, we are going to implement two methods for data debiasing: [Fair PCA](https://deepai.org/publication/efficient-fair-pca-for-fair-representation-learning) and [A Geometric Solution to Fair Representations](https://dl.acm.org/doi/10.1145/3375627.3375864).
3. We are going to evaluate the performance on two sensitive features: `SEX` (i.e. *Males* and *Females*) and `RAC1P` (we will consider only *Whites* and *African-Americans*)
4. I updated the filtering method `adult_filter` to keep the specified groups.

In [ ]:
data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
acs_data = data_source.get_data(states=["CA"], download=True)

def adult_filter(data):
    """Mimic the filters in place for Adult data.
    Adult documentation notes: Extraction was done by Barry Becker from
    the 1994 Census database. A set of reasonably clean records was extracted
    using the following conditions:
    ((AAGE>16) && (AGI>100) && (AFNLWGT>1)&& (HRSWK>0))
    """
    df = data
    df = df[df['AGEP'] > 16]
    df = df[df['PINCP'] > 100]
    df = df[df['WKHP'] > 0]
    df = df[df['PWGTP'] >= 1]
    df = df[df["RAC1P"] < 3] ## keep only Whites and African-Americans
    return df


ACSIncomeNew = BasicProblem(
    features=[
        'AGEP',
        'COW',
        'SCHL',
        'MAR',
        'CIT',
        'RELP',
        'WKHP',
        'SEX',
        'RAC1P'
    ],
    target='PINCP',
    target_transform=lambda x: x > 25000,    
    group=['SEX', "RAC1P"],
    preprocess=adult_filter,
    postprocess=lambda x: np.nan_to_num(x, -1),
)
# acs_data = acs_data.sample(500, random_state=0)

# 1.2 Proxies

In [ ]:
from sklearn.model_selection import train_test_split
data = ACSIncomeNew.df_to_pandas(acs_data)

X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(
    data[0], data[1], data[2], test_size=0.2, random_state=0)

In [ ]:
import pandas as pd
african_american = X_train[X_train['RAC1P'] == 2]
whites = X_train[X_train['RAC1P'] == 1]

from sklearn.utils import resample
african_american = resample(african_american,
                            replace=True,
                            n_samples=len(whites),
                            random_state=0)
oversampled_df = pd.concat([african_american,whites]).reset_index(drop=True)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# plot feature distributions for both genders in one plot
fig, axes = plt.subplots(7, figsize=(20, 40))
for i, feature in enumerate(['AGEP', 'COW', 'SCHL', 'SEX', 'CIT', 'MAR','WKHP']):
    sns.histplot(data=oversampled_df, x=feature, hue='RAC1P', ax=axes[i], palette='Set2')
plt.show()

In [ ]:
import dython
dython.nominal.associations(X_train, nominal_columns=['COW', 'CIT', 'RAC1P', 'SEX', 'RELP', 'MAR'], mark_columns=True);

# 2.1 Data

In [ ]:
definition_df = data_source.get_definitions(download=True)
categories = generate_categories(features=ACSIncomeNew.features, definition_df=definition_df)
features, labels, groups = ACSIncomeNew.df_to_pandas(acs_data, categories=categories, dummies=True)
### groups now contain information about SEX and RAC1P

In [ ]:
# Drop the "redundant" columns
features = features.drop(["RELP_Unmarried partner",
                          "CIT_U.S. citizen by naturalization",
                          "SEX_Male",
                          "SCHL_1 or more years of college credit, no degree",  
                          "MAR_Divorced", 
                          "RELP_Adopted son or daughter",
                          'COW_Working without pay in family business or farm', 
                          "RAC1P_White alone" ], axis = 1) 

print("Columns with the protected features:")
for i, f in enumerate(features.columns):
    if ("RAC1P" in f) or ("SEX" in f):
        print("Column ID: %s" %i, "(%s)"%f)

In [ ]:
X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(
    features.values, labels.values.reshape(-1), groups, test_size=0.3, random_state=0, shuffle=True)

N = 500 ### I am subsampling because it is slow on my machine

X_train = X_train[:N]
y_train = y_train[:N]
group_train = group_train[:N]
X_test = X_test[:N]
y_test = y_test[:N]
group_test = group_test[:N]

In [ ]:
## last columns of our data contains the protected features
protected = X_train[:,56:] 
nonprotected = X_train[:,:56]

In [ ]:
protected

#### Task 2.2.
Use the following arguments in the `opt.fmin_funct`: `xtol=1e-4, ftol=1e-4,  maxfun=1000`

In [ ]:
gammas = np.logspace(1e-5,1e-2,10)
###########
# YOUR CODE
###########

In [ ]:
def sigmoid(x):
    """
    This is logistic regression
    f = 1/(1+exp(-beta^T * x))
    This function assumes as input that you have already multiplied beta and X together
    """
    return 1/(1+np.exp(-x))

def logistic_loss(y_true, y_pred, eps = 1e-9):
    """
    Loss for the logistic regression, y_preds are probabilities
    eps: epsilon for stability
    """
    # check whether that's the same as 1/m * sum()
    return -np.mean(y_true*np.log(y_pred + eps)+(1-y_true)*np.log(1-y_pred + eps))
    
def l2_loss(beta):
    """
    L2-Regularisation
    """
    return np.linalg.norm(beta)

def compute_gradient(beta, X, y_true, _gamma): # y = y_true
    """Calculate the gradient - used for finding the best beta values. 
       You do not need to use groups and lambda (fmin_tnc expects same input as in func, that's why they are included here)"""
    grad = np.zeros(beta.shape)
    m = X.shape[0]
    y_pred = np.array(sigmoid(X.dot(beta))) # same as beta @ X.T?
    
    for i in range(len(grad)):
        if i == 0:
            # don't want to penalize the intercept
            grad[i] = (1/m) * (y_pred - y_true).dot(X[:,i]) 
        else:
            # start with beta[1]
            grad[i] = (1/m) * (y_pred - y_true).dot(X[:,i]) + 2* _gamma *beta[i] 
    #print(f'Gradient: {grad}')
    return grad

def compute_cost(beta , X, y_true, grouping, _lambda, _gamma):
    """Computes cost function with constraints"""
    y_pred = sigmoid(X.dot(beta)) 
    cost = logistic_loss(y_true, y_pred) + _gamma*l2_loss(beta[1:]) # to not penalize the Intercept?
    #print(f'Cost: {cost}')
    return cost

beta = np.random.rand(X_train.shape[1])
# set parameters
lambda_ = 0.1
gamma_ = 0.1
#
X = X_train
y = y_train
groups = group_train

beta = np.random.rand(58)
lambdas = np.linspace(1,1e5, 20)

########## This is the optimisation function

result = opt.fmin_tnc(func=compute_cost, x0=beta, approx_grad = True, maxfun = 1000,
                         args = (nonprotected,y_train, groups, lambda_, gamma_), ftol=1e-4, xtol=1e-4)

In [ ]:
from sklearn.model_selection import KFold

# Define a list of values for lambda to try
lambdas = np.linspace(1, 1e5, 20)

# Initialize an array to store the cross-validation errors for each lambda value
cv_errors = np.zeros(len(lambdas))

# Define the number of folds for K-fold cross-validation
num_folds = 5

# Create the KFold cross-validation object
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Loop over each lambda value
for i, lambda_val in enumerate(lambdas):
    
    # Initialize an array to store the prediction errors for each fold
    fold_errors = np.zeros(num_folds)
    
    # Loop over each fold
    for j, (train_idx, val_idx) in enumerate(kf.split(X_train)):
        
        # Split the data into training and validation sets for this fold
        X_fold_train, y_fold_train = X_train[train_idx], y_train[train_idx]
        X_fold_val, y_fold_val = X_train[val_idx], y_train[val_idx]
        
        # Train the logistic regression model with L2-regularization on the training data for this fold
        beta = np.random.rand(X_train.shape[1])
        result = opt.fmin_tnc(func=compute_cost, x0=beta, approx_grad=True, maxfun=1000,
                              args=(X_fold_train, y_fold_train, group_train, lambda_val, gamma_), 
                              ftol=1e-4, xtol=1e-4)
        beta_opt = result[0]
        
        # Make predictions on the validation data and calculate the prediction error
        y_pred = sigmoid(X_fold_val.dot(beta_opt))
        fold_errors[j] = logistic_loss(y_fold_val, y_pred)
        
    # Calculate the average prediction error for this lambda value
    cv_errors[i] = np.mean(fold_errors)
    
# Find the lambda value with the lowest cross-validation error
best_lambda = lambdas[np.argmin(cv_errors)]

# Train the logistic regression model on the entire training data with the best lambda value
beta = np.random.rand(X_train.shape[1])
result = opt.fmin_tnc(func=compute_cost, x0=beta, approx_grad=True, maxfun=1000,
                      args=(X_train, y_train, group_train, best_lambda, gamma_), 
                      ftol=1e-4, xtol=1e-4)
beta_opt = result[0]

# Make predictions on the test data
y_pred_test = sigmoid(X_test.dot(beta_opt))
test_error = logistic_loss(y_test, y_pred_test)


#### Task 2.3
Use the following arguments in the `opt.fmin_funct`: ` xtol=1e-3, ftol=1e-3, approx_grad=True, maxfun=1000`

In [ ]:
lambdas = np.array([1e-3, 5e-3, 1e-2, 5e-2, 0.1, 1])
###########
# YOUR CODE
###########

In [ ]:
### CODE FROM HANNAH
def sigmoid(x):
    """
    This is logistic regression
    f = 1/(1+exp(-beta^T * x))
    This function assumes as input that you have already multiplied beta and X together
    """
    return 1/(1+np.exp(-x))

def logistic_loss(y_true, y_pred, eps = 1e-9):
    """
    Loss for the logistic regression, y_preds are probabilities
    eps: epsilon for stability
    """
    # check whether that's the same as 1/m * sum()
    return -np.mean(y_true*np.log(y_pred + eps)+(1-y_true)*np.log(1-y_pred + eps))
    
def l2_loss(beta):
    """
    L2-Regularisation
    """
    return np.linalg.norm(beta)

def fair_loss(y_true, y_pred, grouping):
    """
    Group fairness Loss
    """
    n = y_true.shape[0]
    
    # SEX
    n1 = np.sum(grouping['SEX'] == 1)
    n2 = n - n1

    cost_sex = 0
    for i in range(n):
        for j in range(i+1, n):
            if y_true[i] == y_true[j]:
                if grouping['SEX'].iloc[i] != grouping['SEX'].iloc[j]:
                    cost_sex += (y_pred[i] - y_pred[j])**2                
    cost_sex = cost_sex/(n1*n2)
    # print(cost_sex)
    
    # RAC1P
    n1 = np.sum(grouping['RAC1P'] == 1)
    n2 = n - n1

    cost_race = 0
    for i in range(n):
        for j in range(i+1, n):
            if y_true[i] == y_true[j]:
                if grouping['RAC1P'].iloc[i] != grouping['RAC1P'].iloc[j]:
                    cost_race += (y_pred[i] - y_pred[j])**2                
    cost_race = cost_race/(n1*n2)
    # print(cost_race)
    
    # Is this right? Or how should the two fairness constraints be connected before applying the scaling?
    return cost_sex + cost_race


### not needed (?)
def compute_gradient(beta,X,y_true,_gamma): # y = y_true
    """Calculate the gradient - used for finding the best beta values. 
       You do not need to use groups and lambda (fmin_tnc expects same input as in func, that's why they are included here)"""
    grad = np.zeros(beta.shape)
    m = X.shape[0]
    y_pred = np.array(sigmoid(X.dot(beta))) # same as beta @ X.T?
    
    for i in range(len(grad)):
        if i == 0:
            # don't want to penalize the intercept
            grad[i] = (1/m) * (y_pred - y_true).dot(X[:,i]) 
        else:
            # start with beta[1]
            grad[i] = (1/m) * (y_pred - y_true).dot(X[:,i]) + 2* _gamma *beta[i] 
    #print(f'Gradient: {grad}')
    return grad


def compute_cost(beta , X, y_true, grouping, _lambda, _gamma):
    """Computes cost function with constraints"""
    y_pred = sigmoid(X.dot(beta)) 
    cost = logistic_loss(y_true, y_pred) + _lambda*fair_loss(y_true, y_pred, grouping) + _gamma*l2_loss(beta[1:]) # to not penalize the Intercept?
    #print(f'Cost: {cost}')
    return cost

In [ ]:
'''np.random.seed(0)
lambdas = np.array([1e-3, 5e-3, 1e-2, 5e-2, 0.1, 1])
betas = np.random.rand(X_train.shape[1])

opt.fmin_tnc(func=compute_cost, x0=beta, approx_grad = True, maxfun = 1000,
                         args = (nonprotected,y_train, groups, lambdas[0], gammas[2]), ftol=1e-3, xtol=1e-3)'''

### Task 2.4: Fair PCA 

In [ ]:
definition_df = data_source.get_definitions(download=True)
#categories = generate_categories(features=ACSIncomeNew.features, definition_df=definition_df)
features, labels, groups = ACSIncomeNew.df_to_pandas(acs_data, categories=categories, dummies=True)

# Drop the "redundant" columns
features = features.drop(["RAC1P_White alone", 
                          "SEX_Male", 
                          "SCHL_1 or more years of college credit, no degree",  
                          "MAR_Divorced", 
                          "RELP_Adopted son or daughter",
                          'COW_Working without pay in family business or farm' ], axis = 1) 

print("Columns with the protected features:")
for i, f in enumerate(features.columns):
    if ("RAC1P" in f) or ("SEX" in f):
        print("Column ID: %s" %i, "(%s)"%f)

### groups now contain information about SEX and RAC1P
X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(
    features.values, labels.values.reshape(-1), groups, test_size=0.3, random_state=0, shuffle=True)

N = 500 ### I am subsampling because it is slow on my machine

X_train = X_train[:N]
y_train = y_train[:N]
group_train = group_train[:N]
X_test = X_test[:N]
y_test = y_test[:N]
group_test = group_test[:N]

In [ ]:
protected = X_train[:,-2:]
nonprotected = X_train[:,:-2]

In [ ]:
features.columns

In [ ]:
# pca 
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# normalize first two columns for age and wkhp
non_protected_features_scaled = nonprotected.copy()
non_protected_features_scaled[:,:2] = scaler.fit_transform(nonprotected[:,:2])


pca = PCA(n_components=len(features.columns)-4) # - protected features
X_pca = pca.fit_transform(non_protected_features_scaled)

# pca.explained_variance_ratio_
# scree plot bar plot
plt.bar(range(1,len(pca.explained_variance_ratio_)+1), pca.explained_variance_ratio_*100)
for i in range(len(pca.explained_variance_ratio_)):
    plt.text(i+0.7, pca.explained_variance_ratio_[i]*100+0.3, str(round(pca.explained_variance_ratio_[i]*100,2))+'%')
plt.xlabel('Principal Component')

In [ ]:
# plot the first three components interactive 3d
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure(data=[go.Scatter3d(
    x=X_pca[:,0],
    y=X_pca[:,1],
    z=X_pca[:,2],
    mode='markers',
    marker=dict(
        size=5,
        color=['red' if x else 'blue' for x in y_train], # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
)])
fig.show()

In [ ]:
# corr matrix comparing protected
import seaborn as sns
import pandas as pd

df = pd.DataFrame(np.concatenate((protected, X_pca), axis=1))

corr = round(df.corr(),2)
corr.columns=['SEX', 'RAC1P', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10', 'PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16', 'PC17', 'PC18', 'PC19', 'PC20', 'PC21', 'PC22', 'PC23', 'PC24', 'PC25', 'PC26', 'PC27', 'PC28', 'PC29', 'PC30', 'PC31', 'PC32', 'PC33', 'PC34', 'PC35', 'PC36', 'PC37', 'PC38', 'PC39', 'PC40', 'PC41', 'PC42', 'PC43', 'PC44', 'PC45', 'PC46', 'PC47', 'PC48', 'PC49', 'PC50', 'PC51', 'PC52', 'PC53', 'PC54', 'PC55', 'PC56']
sns.heatmap(corr,
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values,
            cmap='RdBu_r',
            # annot=True,
            linewidth=0.5);

print(f'WE SEE THAT THERE ARE NO CORRELATION BETWEEN ANY OF THE PCA COMPONENTS, WHICH MAKES SENSE SINCE PRINCIPAL COMPONENTS TRY TO COVER DIFFERENT ORTHAGONAL DIRECTIONS AND VARIANCE IN THE DATA.')
print(f'Also Sex is slightly correlated PC3 and PC4, which can mean that they are somewhat proxies.')

## 2.4.2

In [ ]:
# project test data with pca
nonprotected_test = X_test[:,:-2]
non_protected_features_scaled_test = nonprotected_test.copy()

# normalize first two columns for age and wkhp
non_protected_features_scaled_test[:,:2] = scaler.transform(nonprotected_test[:,:2])
X_pca_test = pca.transform(non_protected_features_scaled_test)

# project it back into the original space
X_reconstructed = pca.inverse_transform(X_pca)
X_reconstructed_test = pca.inverse_transform(X_pca_test)

# calculate reconstruction error for each sample as mean absolute error
reconstruction_error = []
for i in range(len(non_protected_features_scaled)):
    reconstruction_error.append(np.mean(np.abs(non_protected_features_scaled[i] - X_reconstructed[i])))
reconstruction_error_test = []
for i in range(len(non_protected_features_scaled_test)):
    reconstruction_error_test.append(np.mean(np.abs(non_protected_features_scaled_test[i] - X_reconstructed_test[i])))

# get reconstruction error for protected features
female_error = []
male_error = []
whites_error = []
african_american_error = []
for i in range(len(reconstruction_error_test)):
    # person is always member of both protected groups

    # 1 is male and 2 is female
    if protected[i][0] == 1:
        male_error.append(reconstruction_error_test[i])
    else:
        female_error.append(reconstruction_error_test[i])

    # 1 is white and 2 is african american
    if protected[i][1] == 1:
        whites_error.append(reconstruction_error_test[i])
    else:
        african_american_error.append(reconstruction_error_test[i])

# calculate mean reconstruction error for each group
print(np.mean(male_error))
print(np.mean(female_error))
print(np.mean(whites_error))
print(np.mean(african_american_error))


## 2.4.3 Fair PCA

In [ ]:
import scipy
X = non_protected_features_scaled.copy()
# create Matrix Z with protected features
Z = protected.copy()
# remove mean from each column
Z = Z - np.mean(Z, axis=0)
# find orthonormal null space spanned by ZTX with scipy.linalg.null_space
R = scipy.linalg.null_space(Z.T @ X)

# Find orthonormal eigenvectors RTXTXR with scipy.linalg.eig
eigvals, eigvecs = scipy.linalg.eig(R.T @ X.T @ X @ R)
# sort eigenvectors by eigenvalues
idx = eigvals.argsort()[::-1]
eigvals = eigvals[idx]
eigvecs = eigvecs[:,idx]

# get matrix of first 50 eigenvectors
L = eigvecs[:,:5]

# projection matrix U = RL
U = R @ L

# project data
X_projected = X @ U


In [ ]:
X_projected

In [ ]:

# see if X_projected is correlated with Z as heatmap
sns.heatmap(np.corrcoef(X_projected.T, Z.T), cmap='RdBu_r', linewidth=0.5);

In [ ]:
# project test data
X_test_projected = non_protected_features_scaled_test @ U

# reproject it back into the original space
X_reconstructed = X_projected @ U.T
X_reconstructed_test = X_test_projected @ U.T

# calculate reconstruction error for each sample as mean absolute error
reconstruction_error = []
for i in range(len(non_protected_features_scaled)):
    reconstruction_error.append(np.mean(np.abs(non_protected_features_scaled[i] - X_reconstructed[i])))
reconstruction_error_test = []
for i in range(len(non_protected_features_scaled_test)):
    reconstruction_error_test.append(np.mean(np.abs(non_protected_features_scaled_test[i] - X_reconstructed_test[i])))
# get reconstruction error for protected features
female_error = []
male_error = []
whites_error = []
african_american_error = []
for i in range(len(reconstruction_error_test)):
    # person is always member of both protected groups

    # 1 is male and 2 is female
    if protected[i][0] == 1:
        male_error.append(reconstruction_error_test[i])
    else:
        female_error.append(reconstruction_error_test[i])

    # 1 is white and 2 is african american
    if protected[i][1] == 1:
        whites_error.append(reconstruction_error_test[i])
    else:
        african_american_error.append(reconstruction_error_test[i])

# calculate mean reconstruction error for each group
print(np.mean(male_error))
print(np.mean(female_error))
print(np.mean(whites_error))
print(np.mean(african_american_error))

In [ ]:
# fit own implementation of logistic regression on projected data

beta = np.random.rand(5)

result = opt.fmin_tnc(func=compute_cost, x0=beta, approx_grad = True, maxfun = 1000,
                         args = (X_projected, y_train, groups, lambda_, gamma_), ftol=1e-4, xtol=1e-4)

In [ ]:
# evaluate on test data by multiplying with result vector
y_pred = X_test_projected @ result


